# Greither unit index, a Sagemath implementation
Here we will implement some of the calculation from the article of Cornelius Greither: *“Improving Ramachandra’s and Levesque’s unit index”*.  
In particular we will evaluate the index $i_\beta$ in a very efficient way and the generators $\xi_a(\beta)$ of the group $C_\beta$

## Definition of $n$ end its factorization

In [1]:
n=42*2
if n%4==2:
    print('Error, n not accepted')
else:
    fn = factor(n)
    show(fn)

2^2 * 3 * 7

Also we define these simple functions that we will use later to have a simple overview of the quatity used.

In [2]:
def p(i):
    return fn[i-1][0]
def e(i):
    return fn[i-1][1]
def pe(i):
    return p(i)**e(i)

In [3]:
pe(1)

4

### Definition of the Power set $P_S$

In [4]:
s = len(fn)
S = [ i +1 for i in range(s)]
PS = Subsets(S).list()
PS.remove(Set(S))

In [5]:
PS

[{}, {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]

`nI(I)` evaluate the quantity $n=\prod_{i\in I} p_i^{e_i}$

In [6]:
def nI(I):
    ret = 1
    for i in I:
        ret *= pe(i)
    return ret

In [7]:
nI({1,3})

28

## Definition of $\zeta$ and the group $G$
Here we define the Cyclotomic Field $\mathbb{Q}(\zeta_n)$ and the variable `z` is assigned to $\zeta_n$.

In [8]:
Qn.<z>=CyclotomicField(n)
(Qn)

Cyclotomic Field of order 84 and degree 24

We could directly evaluate the group $G_0$ as the Galois group of the Field, but it uses a long computational time, so we will follows a different approach: defining $G$ directly from its elements.  
Also I have done some functions to do some operations in $G$, but in the end I have not used them

In [9]:
G_0 = Qn.galois_group() #loooooong time
G_0.an_element()

(1,20,3,22,5,24)(2,21,4,23,6,19)(7,14,9,16,11,18)(8,15,10,17,12,13)

In [10]:
G = [ a for a in [1..n//2] if gcd(a,n)==1]

In [11]:
show(G)

[1, 5, 11, 13, 17, 19, 23, 25, 29, 31, 37, 41]

In [12]:
def corr(r):
    if r not in G:
        r = n-r
    return r

def power(x,a):
    r = power_mod(x,a,n)
    return corr(r)

def molt(x,y):
    r = mod(x*y,n)
    return corr(r)

In [13]:
molt(5,11)

29

# Calculation of the index
First we define the number field $K$ as the Maximal Real subfield of $\mathbb{Q}(\zeta_n)$, using that is equal to $\mathbb{Q}(\zeta_n + \zeta_n^{-1})$

In [14]:
zz = z + z.conjugate()

In [15]:
K.<zc> = NumberField(zz.minpoly(),'zc')

We define $\epsilon_i$ knowing that is equal to $\phi(p_i^{e_i})$ and $g_i$ using its definition: we embed the the prime $p_i$ in $O_K$ and we factor it

In [16]:
def eps(i):
    return euler_phi(pe(i))

In [17]:
def g(i):
    I = K.ideal(p(i))
    return len(I.factor())

For $f_i$ we use the equality $[K:\mathbb{Q}] = \epsilon_i g_i f_i $ since we know the other three elements.  
*Memo*: $[K:\mathbb{Q}] = \phi(n)/2 $  
Later we will see another possible evaluation (a bit slower) using the Frobenious morphism. 

In [18]:
def f(i):
    return euler_phi(n)/(2*g(i)*eps(i))

So we have that:

In [19]:
def i_b():
    i_b=1
    for i in S:
        i_b *= (eps(i)**(g(i)-1)) * (f(i)**(2*g(i) -1))
    return i_b

In [20]:
i = i_b()
i

648

In [21]:
show(factor(i))

2^3 * 3^4

Here we have another evaluation of $i_\beta$ that compress all the calculation to optimize the result

In [22]:
def i_b_compressed(n):
    fn = factor(n)
    s = len(fn)
    S = [ i +1 for i in range(s)]
    K.<z> = CyclotomicField(n)
    zz = z + z.conjugate()
    K = NumberField(zz.minpoly(),'a')
    ibb=1
    for j in S:
        eps = euler_phi(fn[j-1][0]**fn[j-1][1])
        g = len(K.ideal(fn[j-1][0]).factor())
        f = euler_phi(n)/(2*g*eps)
        ibb *= (eps**(g-1)) * (f**(2*g-1))
    return ibb

In [47]:
show(factor(i_b_compressed(5*7*11)))

2^6 * 3^8 * 5^5

In [48]:
2**6 * 3**8 * 5**5

1312200000

If we do the evaluation with Ramachandra's unit group we get:
$$ i_\beta = 968 84935 06496 61515 61530 58565 67725 19049 20520 00000 $$

In [50]:
show(factor(968849350649661515615305856567725190492052000000))

2^8 * 3^2 * 5^6 * 7^5 * 11^2 * 19^4 * 31 * 61 * 71 * 181 * 191 * 281 * 521 * 1741 * 2801 * 4021 * 7621

# Generators construction
Now with several steps we proceed in the costruction of the generators, starting with the definition of $\beta$

## Definition the Frobenious morphism
Given $i \in S$ we want to find a lift in $G_0$ of the frobenious morphism:

\begin{alignat*}{2}
		F_i : \mathbb{Q}(\zeta_{n/p_i^{e_i}})^+ &\longrightarrow \: \mathbb{Q}(\zeta_{n/p_i^{e_i}})^+  \\
		\zeta_{n/p_i^{e_i}}  &\longmapsto \: \zeta_{n/p_i^{e_i}} ^ {p_i}
\end{alignat*} 
So we need an element $f$ that sends $\zeta_{n/p_i^{e_i}} \simeq \zeta _n^{p_i^{e_i}}$ in $\zeta _n^{p_i^{e_i+1}}= \zeta _n^{p_i^{e_i} p_i}$

First we can see it as an integer in the list `G`

In [24]:
def frob(i):
    zi = z^pe(i)
    for f in G:
        if zi^f==zi^p(i):
            return f

But also we can give the result directly as an automorphism of $\mathbb{Q}(\zeta_{n})$ (so an element in $G_0$).  
**Remark**: With this costruction we directly considerate a lifting of the function without defining it on the field $\mathbb{Q}(\zeta_{n/p_i^{e_i}})^+$.  
*Memo:* We have seen that the final results does not depends on the particular lifting

In [25]:
def frobhom(i):
    zi = z^pe(i)
    for f in G:
        if zi^f==zi^p(i):
            return Qn.hom([z**f])

In [26]:
(frobhom(1))

Ring endomorphism of Cyclotomic Field of order 84 and degree 24
  Defn: z |--> z^23

Now to find the trace elements we need to have the order of the element $f$ in $G_i$.  
We already know that this is the inertia degree of the prime $p_i$, but now we will follow a different approach.
Also here we define the morphism on $\mathbb{Q}(\zeta_{n/p_i^{e_i}})$ and we check when its generator (`zz`) is fixed.  
**Remark**: is not the same of checking when $\zeta$ is fixed

In [27]:
def ford(i):
    Qi.<zi>=CyclotomicField(n/pe(i))
    f = Qi.hom([zi^p(i)])
    o = 1
    zz=zi+zi.conjugate()
    while (not (f**o)(zz)==zz) : 
        o += 1
    return o

We can see that the two results are indeed equivalent

In [28]:
[ford(i)==f(i) for i in S]

[True, True, True]

## Definition of $\beta$ and its evaluation  
We start defining its value on the singletons $\{i\}$. Also to have an elements in $\mathbb{Z}[G_0]$ we simply use a list of elements in $G_0$, since we only need to evalutate them and we can simply use a recursive evalutation.  
**Remark**: we use `f(i)` instead of `ford(i)` because it is faster

In [29]:
def beta0(i : int):
    fi = frobhom(i)
    return [ fi**j for j in range(f(i))]

We anticipate the valutations on the singletons to save computational time later

In [30]:
vbeta = [beta0(i) for i in S]

For our costruction the only thing we need is $\zeta ^{ \beta(I)}$ (`valbeta(base,I)`), that we can evaluate starting from $\zeta ^{ \beta(i)}$ (`valbeta0(base,i)`) using that $\beta(I) = \prod_{i \in S} \beta(i)$ and $\zeta^{\gamma \delta} = (\zeta^\gamma )^\delta$

In [31]:
def valbeta0(base,i):
    v = 1
    for vf in vbeta[i-1]:
        v *= vf(base)
    return v

In [32]:
def valbeta(base,I):
    if I.is_empty():
        return base
    for i in I:
        base = valbeta0(base,i)
    return base

# Calculation of the generators
We proceed now with the evalutation of
	$$ z(\beta ):= \prod_{I \in P_S } z_I ^{\beta(I)} $$ 
where $ z_I  := 1 - \zeta ^{n_I}$

In [33]:
def zbeta():
    ret = 1
    for I in PS:
        zI= 1 - z**nI(I)
        ret *= valbeta(zI,I)
    return ret

We store it in the memory to save computational time later, also we can see that this is a very long and complicate element

In [34]:
zb = zbeta()

In [35]:
show(zb)

4000752*z^23 + 4000752*z^22 + 2000376*z^21 - 2000376*z^19 - 4000752*z^18 - 4000752*z^17 + 4000752*z^12 + 2000376*z^11 + 2000376*z^10 - 4000752*z^9 - 2000376*z^8 - 4000752*z^7 - 2000376*z^6 - 2000376*z^5 + 2000376*z^4 + 2000376*z^2 + 2000376*z + 4000752

An interesting wall of numbers, however now we can use it to evaluate the elements 
$$ \zeta ^{\frac{(1-a)}{2} n_I \beta (I)} $$ 
and
\begin{equation}
		\zeta ^{(1-a)\frac{t}{2}} \text{ with } t = \sum_{I \in P_S} n_I \beta(I) 
	\end{equation}

In [36]:
def zdbeta(a,I):
    d = (1-a)*nI(I)/2
    valbeta(z^d,I)

In [37]:
def zdbeta(a):
    ret = 1
    d= (1-a)/2
    for I in PS:
        ret *= valbeta(z^(d*nI(I)),I)
    return ret

And finally we can evaluate
	\begin{equation}
		\xi_a (\beta) := \zeta ^{d_a (\beta)} \frac{\sigma_a (z(\beta))}{z(\beta)} \text{ with } d_a(\beta)= (1-a)\frac{t}{2}
	\end{equation}

In [38]:
def xibeta(a):
    sa = Qn.hom([z**a])
    return zdbeta(a)*sa(zb)/zb

Consider a particulare case, we can easly see that the result is in the ring of integer $\mathbb{Z}[\zeta]$ and that it is real comparing itself with the conjugate

In [39]:
a = G[3]
xi = xibeta(a)

In [40]:
show(xi)

11*z^23 + 11*z^22 + 20*z^21 + 20*z^20 + 20*z^19 + 14*z^18 + 3*z^17 + 3*z^16 - 14*z^15 - 11*z^14 - 14*z^13 - 14*z^12 - 3*z^11 - 3*z^10 - 3*z^9 - 9*z^8 - 12*z^7 - 20*z^6 - 31*z^5 - 25*z^4 - 31*z^3 - 25*z^2 - 14*z - 14

In [41]:
xi.conjugate()==xi

True

Without looking to the the monomials we can also direclty look for coefficents not in $\mathbb{Z}$

In [42]:
[i for i in range(euler_phi(n)) if not (xi[i] in ZZ) ]

[]

Infact the list is empty

In [43]:
show(xi^(-1))

28*z^23 + 12*z^22 + 6*z^21 - 20*z^20 - 34*z^19 - 14*z^18 - 14*z^17 + 22*z^16 + 14*z^15 + 11*z^14 + 3*z^13 - 11*z^12 + 14*z^11 + 3*z^10 - 11*z^9 - 14*z^8 - 29*z^7 - 3*z^6 + 6*z^5 + 25*z^4 + 31*z^3 + 3*z - 22

# Calculation of the class number

We create the group:
$$ C_\beta = \left< -1 , \xi_\beta^a , \text{ for } a \in G \right> $$

In [ ]:
C = [xibeta(a) for a in G[1:]]

In [ ]:
C.append(-(xi^0))

We consider the maximal real subfield $K$ and its unit group

In [ ]:
KK = Qn.subfield(zz)[0]
KK

In [ ]:
EK = UnitGroup(KK)

In [ ]:
We check that it contains the units

In [ ]:
[C[i] in EK for i in range(len(C)) ]

We evaluate the subgroup of $E_K$ and its index (the cardinality of $E_K / C_\beta$)

In [ ]:
Cb = EK.subgroup(C)

In [ ]:
H = EK.quotient(Cb)

We use that 
$$ [E_K:C_\beta]=h_K i_\beta $$
to finf the **Class number**

In [ ]:
H.cardinality()/i